In [5]:
'''Import necessary packages'''
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
import seaborn as sns
import numpy as np
import pandas as pd

%matplotlib inline

In [6]:
path = '/Users/yanjingding/Downloads/NEU/bigdata/asignment3/'

In [7]:
data = pd.read_csv(f'{path}/BlackFriday.csv')
print(data.shape)
data.head()

(537577, 12)


,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


# Preprocess the data

In [8]:
# Identify ID and Categorical columns
ID_col = ['User_ID', 'Product_ID']
flag_col = ['Type']
target_col = ['Purchase']
cat_cols = ['Gender', 'Age', 'City_Category', 'Stay_In_Current_City_Years']
num_cols = list(set(list(data.columns))-set(cat_cols)-set(ID_col)-set(target_col)-set(flag_col))
print (num_cols)

['Occupation', 'Product_Category_3', 'Product_Category_1', 'Product_Category_2', 'Marital_Status']


In [9]:
# Combine numerical and Categorical variables
num_cat_cols = num_cols+cat_cols

In [10]:
# Flag missing values
for var in num_cat_cols:
    if data[var].isnull().any() == True:
        data[var+'_NA'] = data[var].isnull() * 1

In [11]:
# Fill numerical missing data with mean
data[num_cols] = data[num_cols].fillna(data[num_cols].mean())

In [12]:
# Fill categorical missing values with -9999
data[cat_cols] = data[cat_cols].fillna(value = -9999)

# Create label encoders for categorical features
from sklearn.preprocessing import LabelEncoder

for var in cat_cols:
    number = LabelEncoder()
    data[var] = number.fit_transform(data[var].astype('str'))
    
# Normalize the data
features = list(set(list(data.columns))-set(ID_col)-set(target_col))
data[features] = data[features] / data[features].max()

In [13]:
features = list(set(list(data.columns))-set(ID_col)-set(target_col)-set(flag_col))

X = data[features]
Y = data[target_col]

In [14]:
# Split the data into 3 parts
(X_train, X_test, Y_train, Y_test) = train_test_split(X, Y, test_size=0.15, random_state=1)
(X_train, X_valid, Y_train, Y_valid) = train_test_split(X_train, Y_train, test_size=0.15, random_state=1)

# Use DNN to predict the price

In [15]:
# Build the deep neural network
model = Sequential()

# input layer
model.add(Dense(100, activation='relu', input_dim=11))

# middle layer
model.add(Dense(50, activation='relu'))

# output layer
model.add(Dense(1))

In [16]:
# Compile the DNN
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy', 'mean_squared_error'])

In [17]:
# Fit the DNN with your train data
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), nb_epoch=10, batch_size=500)

/Users/yanjingding/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app


Train on 388399 samples, validate on 80637 samples
Epoch 1/10
388399/388399 [==============================] - 4s 10us/step - loss: 64385110.9660 - acc: 4.3769e-05 - mean_squared_error: 64385110.9660 - val_loss: 30169662.2265 - val_acc: 1.2401e-05 - val_mean_squared_error: 30169662.2265

In [18]:
# Evaluate the model
from sklearn.metrics import mean_squared_error
pred = model.predict(X_valid)
score = np.sqrt(mean_squared_error(Y_valid, pred))
print('NN Score:', score)

NN Score: 4596.636587299247
